In [ ]:
# 0) Parâmetros globais (edite aqui)
import os

# Caminhos
DATA_PATH = os.getenv('DATA_PATH', '/content/drive/MyDrive/FineTunning/TechChallenge03/trn.json.gz')
ART_DIR = os.getenv('ART_DIR', '/content/drive/MyDrive/FineTunning/TechChallenge03/artifacts')
OUTPUT_DIR = os.getenv('OSS_OUTPUT_DIR', '/content/drive/MyDrive/FineTunning/TechChallenge03/tinyllama_amazon_finetuned')

# Semente
SEED = int(os.getenv('SEED', 42))

# Leitura/Limpeza
MAX_RECORDS = int(os.getenv('MAX_RECORDS_FOR_FT', 200_000))
TITLE_MIN_LEN = int(os.getenv('TITLE_MIN_LEN', 2))
CONTENT_MIN_LEN = int(os.getenv('CONTENT_MIN_LEN', 5))

# Dataset/Splits
TEST_SIZE = float(os.getenv('TEST_SIZE', 0.2))
VAL_SIZE = float(os.getenv('VAL_SIZE', 0.5))

# Modelo e geração
BASE_OSS_MODEL = os.getenv('BASE_OSS_MODEL', 'TinyLlama/TinyLlama-1.1B-Chat-v1.0')
OSS_MAX_SEQ_LEN = int(os.getenv('OSS_MAX_SEQ_LEN', 1024))
OSS_SUBSET = int(os.getenv('OSS_SUBSET', 4000))
OSS_EVAL_N = int(os.getenv('OSS_EVAL_N', 200))
OSS_TEST_N = int(os.getenv('OSS_TEST_N', 200))
GEN_MAX_NEW_TOKENS = int(os.getenv('GEN_MAX_NEW_TOKENS', 96))
GEN_BATCH_SIZE = int(os.getenv('GEN_BATCH_SIZE', 8))
GEN_DO_SAMPLE = bool(int(os.getenv('GEN_DO_SAMPLE', 1)))
# test

# Treino
OSS_LR = float(os.getenv('OSS_LR', 2e-4))
OSS_BATCH_SIZE = int(os.getenv('OSS_BATCH_SIZE', 4))
OSS_GRAD_ACCUM = int(os.getenv('OSS_GRAD_ACCUM', 2))
OSS_EPOCHS = int(os.getenv('OSS_EPOCHS', 2))
OSS_WARMUP_STEPS = int(os.getenv('OSS_WARMUP_STEPS', 50))

# LoRA
LORA_R = int(os.getenv('LORA_R', 16))
LORA_ALPHA = int(os.getenv('LORA_ALPHA', 16))
LORA_DROPOUT = float(os.getenv('LORA_DROPOUT', 0.05))

print('Parâmetros carregados:')
print({k:v for k,v in globals().items() if k.isupper()})

# Tech Challenge (Colab GPU): Fine-tuning OSS com Unsloth + TinyLlama

Pipeline completo para rodar no Google Colab com GPU (sem custo de API):

1) Montar Google Drive e instalar dependências
2) Carregar e limpar dados (title → content) a partir de `/content/drive/MyDrive/FineTunning/TechChallenge03/trn.json.gz`
3) Preparar dataset para SFT (prompts + splits)
4) Avaliação baseline (modelo base, sem FT)
5) Treinamento LoRA com Unsloth + TinyLlama
6) Avaliação pós-FT e gráficos
7) Salvar artefatos (adapter, métricas, amostras) no Drive

Observações:
- Use GPU (Runtime > Change runtime type > T4/L4/A100).
- bitsandbytes (4-bit) ajuda caber na VRAM.
- Ajuste OSS_SUBSET/OSS_EVAL_N/OSS_EPOCHS conforme tempo e GPU.

In [ ]:
# 1) Montar Drive e instalar dependências (Colab)
import sys, subprocess, os

IN_COLAB = False
try:
    import google.colab  # type: ignore
    IN_COLAB = True
except Exception:
    pass

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

PKGS = [
    'unsloth>=2024.8.12',
    'transformers>=4.43',
    'accelerate>=0.33',
    'datasets>=2.20',
    'peft>=0.12.0',
    'trl>=0.9.6',
    'sacrebleu',
    'rouge-score',
    'matplotlib',
    'tiktoken'
]

print('Instalando dependências...')
subprocess.check_call([sys.executable, '-m', 'pip', 'install', *PKGS])

# bitsandbytes geralmente já existe no Colab; garantimos
try:
    import bitsandbytes as bnb  # type: ignore
except Exception:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', 'bitsandbytes'])

print('Setup concluído.')

In [ ]:
# 2) Carregar e limpar dados (title → content)
import gzip, json, re
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

assert os.path.exists(DATA_PATH), f'Arquivo não encontrado: {DATA_PATH}'

rows, count = [], 0
with gzip.open(DATA_PATH, 'rt', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            obj = json.loads(line)
        except json.JSONDecodeError:
            continue
        rows.append({'title': obj.get('title'), 'content': obj.get('content')})
        count += 1
        if count >= MAX_RECORDS:
            break

df_raw = pd.DataFrame(rows, columns=['title','content']).dropna(how='all')
print('Lidas linhas:', len(df_raw))

EMPTY_STRINGS = {'none','nan','null','na','n/a'}
def clean_text(s):
    if s is None:
        return None
    if isinstance(s, float) and np.isnan(s):
        return None
    if isinstance(s, str) and s.strip().lower() in EMPTY_STRINGS:
        return None
    s = str(s)
    s = re.sub(r'<[^>]+>', ' ', s)
    s = re.sub(r'[`*_#>\"]', ' ', s)
    s = s.replace('\r', ' ').replace('\n', ' ')
    s = re.sub(r'\s+', ' ', s).strip(" '\"")
    if not s or s.strip().lower() in EMPTY_STRINGS:
        return None
    return s

df = df_raw.copy()
df['title'] = df['title'].apply(clean_text)
df['content'] = df['content'].apply(clean_text)

before = len(df)
df = df.dropna(subset=['title','content']).drop_duplicates(subset=['title','content'])
# Filtros mínimos parametrizados
before_len = len(df)
df = df[df['title'].str.len() >= TITLE_MIN_LEN]
df = df[df['content'].str.len() >= CONTENT_MIN_LEN]
print(f'Após limpeza: {len(df)} (removidos {before-len(df)}; por tamanho {before_len-len(df)})')

In [ ]:
# 3) Preparar dataset para SFT (prompts + splits)
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

SYSTEM_PROMPT = os.getenv('OSS_SYSTEM_PROMPT', 'Você é um assistente que gera descrições detalhadas de produtos a partir do título.')
PROMPT_COL = 'prompt'
TARGET_COL = 'target'

df_prep = df[['title','content']].copy()
df_prep[PROMPT_COL] = df_prep['title'].apply(lambda t: f"[SYSTEM]\n{SYSTEM_PROMPT}\n[USER]\nTítulo: {t}\n[ASSISTANT]\n")
df_prep[TARGET_COL] = df_prep['content']

test_size = float(os.getenv('TEST_SIZE', 0.2))
val_size = float(os.getenv('VAL_SIZE', 0.5))

train_df, temp_df = train_test_split(df_prep[[PROMPT_COL,TARGET_COL]], test_size=test_size, random_state=SEED, shuffle=True)
val_df, test_df = train_test_split(temp_df, test_size=val_size, random_state=SEED, shuffle=True)

print(f"Splits → train={len(train_df):,}, val={len(val_df):,}, test={len(test_df):,}")

datasets_dict = DatasetDict({
    'train': Dataset.from_pandas(train_df.reset_index(drop=True)),
    'validation': Dataset.from_pandas(val_df.reset_index(drop=True)),
    'test': Dataset.from_pandas(test_df.reset_index(drop=True)),
})

datasets_dict

In [ ]:
# 4) Carregar modelo base (Unsloth + TinyLlama) e avaliação baseline (val e teste) — com geração em lote
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
from rouge_score import rouge_scorer
import sacrebleu
import numpy as np
from math import ceil

MAX_SEQ_LEN = OSS_MAX_SEQ_LEN

device = 'cuda' if torch.cuda.is_available() else ('mps' if getattr(torch.backends, 'mps', None) and torch.backends.mps.is_available() else 'cpu')
print('Dispositivo:', device)

print('Carregando modelo base...')
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = BASE_OSS_MODEL,
    max_seq_length = MAX_SEQ_LEN,
    dtype = None,
    load_in_4bit = True if device == 'cuda' else False,
)

# Subconjuntos para acelerar
if datasets_dict['train'].num_rows > OSS_SUBSET:
    datasets_dict['train'] = datasets_dict['train'].shuffle(seed=SEED).select(range(OSS_SUBSET))
subset_val = datasets_dict['validation'].select(range(min(OSS_EVAL_N, datasets_dict['validation'].num_rows)))
subset_test = datasets_dict['test'].select(range(min(OSS_TEST_N, datasets_dict['test'].num_rows)))

model.eval()
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
    model.resize_token_embeddings(len(tokenizer))

# Geração batelada e remoção de prompt precisa via input_length
@torch.no_grad()
def generate_batch(prompts, max_new_tokens=GEN_MAX_NEW_TOKENS, do_sample=GEN_DO_SAMPLE, temperature=0.7, top_p=0.9):
    inputs = tokenizer(prompts, return_tensors='pt', padding=True, truncation=True, max_length=MAX_SEQ_LEN).to(model.device)
    input_lengths = (inputs['input_ids'] != tokenizer.pad_token_id).sum(dim=1)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=do_sample,
        temperature=temperature,
        top_p=top_p,
        pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
    )
    texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    # Remover exatamente os tokens do prompt por amostra
    result = []
    for i, txt in enumerate(texts):
        prompt_text = tokenizer.decode(outputs[i][:input_lengths[i]], skip_special_tokens=True)
        result.append(txt[len(prompt_text):].strip())
    return result

# Helper para avaliação batelada
def eval_dataset(ds, batch_size=GEN_BATCH_SIZE):
    refs, hyps = [], []
    n = ds.num_rows
    for i in range(0, n, batch_size):
        batch = ds.select(range(i, min(i+batch_size, n)))
        prompts = [ex['prompt'] for ex in batch]
        hyps.extend(generate_batch(prompts))
        refs.extend([ex['target'].strip() for ex in batch])
    bleu = sacrebleu.corpus_bleu(hyps, [refs]).score
    scorer = rouge_scorer.RougeScorer(['rougeLsum'], use_stemmer=True)
    rougeL = float(np.mean([scorer.score(r, h)['rougeLsum'].fmeasure for r, h in zip(refs, hyps)]))
    return bleu, rougeL, refs, hyps

# Baseline (val e test)
bleu_base_val, rougeL_base_val, refs_val, hyps_base_val = eval_dataset(subset_val)
print(f'Baseline (val) → BLEU={bleu_base_val:.2f} | ROUGE-L={rougeL_base_val:.3f}')
bleu_base_test, rougeL_base_test, refs_test_base, hyps_base_test = eval_dataset(subset_test)
print(f'Baseline (test) → BLEU={bleu_base_test:.2f} | ROUGE-L={rougeL_base_test:.3f}')

In [ ]:
# 5) Treinamento LoRA (Unsloth + TRL SFTTrainer)
from transformers import TrainingArguments, __version__ as HF_VER
from trl import SFTTrainer
import inspect

LORA_R = int(os.getenv('LORA_R', 16))
LORA_ALPHA = int(os.getenv('LORA_ALPHA', 16))
LORA_DROPOUT = float(os.getenv('LORA_DROPOUT', 0.05))
LR = float(os.getenv('OSS_LR', 2e-4))
BATCH_SIZE = int(os.getenv('OSS_BATCH_SIZE', 4))
GR_ACCUM = int(os.getenv('OSS_GRAD_ACCUM', 2))
EPOCHS = int(os.getenv('OSS_EPOCHS', 2))
WARMUP = int(os.getenv('OSS_WARMUP_STEPS', 50))
OUTPUT_DIR = os.getenv('OSS_OUTPUT_DIR', '/content/drive/MyDrive/FineTunning/TechChallenge03/tinyllama_amazon_finetuned')

# Aplicar LoRA (com Unsloth)
model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_R,
    target_modules=['q_proj','k_proj','v_proj','o_proj','gate_proj','up_proj','down_proj'],
    lora_alpha=LORA_ALPHA,
    lora_dropout=LORA_DROPOUT,
    bias='none',
    use_gradient_checkpointing=True,
)

# Preparar dataset tokenizado

def format_row(ex):
    return {'text': ex['prompt'] + ex['target']}

train_ds = datasets_dict['train'].map(format_row, remove_columns=datasets_dict['train'].column_names)
val_ds = datasets_dict['validation'].map(format_row, remove_columns=datasets_dict['validation'].column_names)

def tokenize(batch):
    return tokenizer(
        batch['text'],
        truncation=True,
        max_length=MAX_SEQ_LEN,
        padding=False,
        return_attention_mask=True,
    )

train_tok = train_ds.map(tokenize, batched=True, remove_columns=['text'])
val_tok = val_ds.map(tokenize, batched=True, remove_columns=['text'])

# Compatibilidade Transformers v4/v5 via inspeção da assinatura
args_kwargs = dict(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GR_ACCUM,
    learning_rate=LR,
    num_train_epochs=EPOCHS,
    warmup_steps=WARMUP,
    bf16=(torch.cuda.is_available() and torch.cuda.is_bf16_supported()),
    fp16=(torch.cuda.is_available() and not torch.cuda.is_bf16_supported()),
    logging_steps=10,
    eval_steps=100,
    save_steps=200,
    save_total_limit=1,
    report_to=[],
)

sig_params = set(inspect.signature(TrainingArguments.__init__).parameters.keys())
if 'eval_strategy' in sig_params:
    args_kwargs['eval_strategy'] = 'steps'
elif 'evaluation_strategy' in sig_params:
    args_kwargs['evaluation_strategy'] = 'steps'
else:
    print('Aviso: parâmetro de avaliação por estratégia não encontrado; usaremos eval_steps para controlar a frequência.')

if 'save_strategy' in sig_params:
    args_kwargs['save_strategy'] = 'steps'

args = TrainingArguments(**args_kwargs)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_tok,
    eval_dataset=val_tok,
    args=args,
    packing=True,
    dataset_text_field=None
)

print('Treinando...')
trainer.train()
print('Treino concluído.')

# Salvar adapter
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)
trainer.save_model(OUTPUT_DIR)
print('Adapter salvo em:', OUTPUT_DIR)

In [ ]:
# 6) Avaliação pós-FT e gráficos + salvamento de métricas (comparando com baseline de teste) — batelada
import matplotlib.pyplot as plt

# Reutiliza subset_test e generate_batch/GEN_BATCH_SIZE definidos no passo 4
model.eval()

def eval_dataset_after_ft(ds, batch_size=GEN_BATCH_SIZE):
    refs, hyps = [], []
    n = ds.num_rows
    for i in range(0, n, batch_size):
        batch = ds.select(range(i, min(i+batch_size, n)))
        prompts = [ex['prompt'] for ex in batch]
        hyps.extend(generate_batch(prompts))
        refs.extend([ex['target'].strip() for ex in batch])
    bleu = sacrebleu.corpus_bleu(hyps, [refs]).score
    scorer = rouge_scorer.RougeScorer(['rougeLsum'], use_stemmer=True)
    rougeL = float(np.mean([scorer.score(r, h)['rougeLsum'].fmeasure for r, h in zip(refs, hyps)]))
    return bleu, rougeL, refs, hyps

bleu_ft, rougeL_ft, refs_test, hyps_ft = eval_dataset_after_ft(subset_test)

print('\nResumo comparativo (TEST):')
print(f"Baseline → BLEU={bleu_base_test:.2f} | ROUGE-L={rougeL_base_test:.3f}")
print(f"Fine-tuned → BLEU={bleu_ft:.2f} | ROUGE-L={rougeL_ft:.3f}")

labels = ['BLEU','ROUGE-L']
baseline_vals = [bleu_base_test, rougeL_base_test]
ft_vals = [bleu_ft, rougeL_ft]

plt.figure(figsize=(6,4))
x = np.arange(len(labels))
width = 0.35
plt.bar(x - width/2, baseline_vals, width, label='Baseline (test)')
plt.bar(x + width/2, ft_vals, width, label='Fine-tuned (test)')
plt.xticks(x, labels)
plt.ylabel('Score')
plt.title('Comparação de métricas no conjunto de teste')
plt.legend()
plt.grid(axis='y', alpha=0.2)
plt.show()

# Salvar métricas e amostras no Drive
os.makedirs(ART_DIR, exist_ok=True)

import json
with open(os.path.join(ART_DIR, 'oss_metrics.json'), 'w', encoding='utf-8') as f:
    json.dump({
        'bleu_base_val': float(bleu_base_val),
        'rougeL_base_val': float(rougeL_base_val),
        'bleu_base_test': float(bleu_base_test),
        'rougeL_base_test': float(rougeL_base_test),
        'bleu_ft_test': float(bleu_ft),
        'rougeL_ft_test': float(rougeL_ft)
    }, f, ensure_ascii=False, indent=2)

samples_path = os.path.join(ART_DIR, 'qualitative_samples_test.jsonl')
with open(samples_path, 'w', encoding='utf-8') as f:
    for ex, r_base, r_ft in zip(subset_test, hyps_base_test, hyps_ft):
        f.write(json.dumps({'prompt': ex['prompt'], 'hyp_base': r_base, 'hyp_ft': r_ft, 'ref': ex['target']}, ensure_ascii=False) + '\n')

print('Artefatos salvos em:', ART_DIR)